In [1]:
import random 
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# print(device)
# import numpy as np
import pandas as pd
import umap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import rdkit as rd
from rdkit import Chem
# import torch.nn as nn
import h5py 
from sklearn import metrics 
import numpy as np

/home/math/20481926/.conda/envs/my-rdkit-env/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/math/20481926/.conda/envs/my-rdkit-env/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/math/20481926/.conda/envs/my-rdkit-env/lib/p

In [2]:
def seed_everything(seed: int):
    r"""Sets the seed for generating random numbers in :pytorch:`PyTorch`,
    :obj:`numpy` and Python.

    Args:
        seed (int): The desired seed.
    """
    random.seed(seed)
    np.random.seed(seed)


seed_everything(0)

In [3]:
df = pd.read_csv('./extend_dataset/ClinTox/ClinTox_extend_v2.csv')

In [4]:
df

,Unnamed: 0,index,A1.0,A1.1,A1.2,A1.3,A1.4,A1.5,A1.6,A1.7,...,E5.121,E5.122,E5.123,E5.124,E5.125,E5.126,E5.127,CT_TOX,smiles,Group
0,169861,FBWNMEQMRUMQSO-UHFFFAOYSA-N,-0.016168,-0.161498,-0.222765,-0.425781,-0.155150,-0.058580,-0.798194,-0.286609,...,-0.243471,-0.081387,-0.140750,0.005702,-0.530805,0.006174,-0.197947,0,CCCCCCCCCc1ccc(cc1)OCCOCCOCCOCCOCCOCCOCCOCCOCCO,1234
1,167683,FAOZLTXFLGPHNG-KNAQIMQKSA-N,-0.384817,0.130702,-0.216485,-0.227028,-0.155242,0.279811,-0.426326,-0.180011,...,-0.106712,0.104000,-0.048428,0.003557,-0.455711,0.241515,-0.253158,0,C[C@H]1C[C@H]2[C@@H]3CC[C@@]([C@]3(C[C@@H]([C@...,530
2,550345,OFZCIYFFPZCNJE-UHFFFAOYSA-N,-0.329531,0.275611,-0.100012,-0.312780,-0.096014,0.037530,-0.752631,0.048236,...,-0.173596,0.219707,0.099544,0.212405,-0.189759,0.227042,-0.073475,0,CCCC(C)(COC(=O)N)COC(=O)NC(C)C,1178
3,0,NaN,0.093270,-0.092842,-0.093273,0.093262,0.093252,-0.093271,-0.092997,-0.093273,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,C[C@@H]1CC[C@H]2C[C@@H](/C(=C/C=C/C=C\\[C@H](C...,433
4,554625,OIRDTQYFTABQOQ-KQYNXXCUSA-N,-0.392563,0.475761,-0.582324,-0.163635,0.144806,-0.330440,-1.123437,0.106033,...,-0.467990,-0.092955,0.038022,-0.118591,-0.642681,0.248162,-0.070433,0,c1nc(c2c(n1)n(cn2)[C@H]3[C@@H]([C@@H]([C@H](O3...,364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,139989,DJHCCTTVDRAMEH-DUUJBDRPSA-N,-0.502756,0.223423,-0.243837,-0.402113,-0.203983,0.022914,-0.737219,0.259320,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,CCC(=O)OCC(=O)[C@]1([C@@H](C[C@@H]2[C@@]1(C[C@...,1131
1473,0,NaN,0.093270,-0.092842,-0.093273,0.093262,0.093252,-0.093271,-0.092997,-0.093273,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,CCn1cc(c(=O)c2c1nc(cc2)C)C(=O)[O-],1295
1474,0,NaN,0.093270,-0.092842,-0.093273,0.093262,0.093252,-0.093271,-0.092997,-0.093273,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,CC/C(=C(\c1ccccc1)/c2ccc(cc2)OCC[NH+](C)C)/c3c...,861
1475,641,AAKJLRGGTJKAMG-UHFFFAOYSA-N,-0.093483,-0.084431,-0.056074,-0.405718,-0.234262,0.166777,-0.568343,-0.487290,...,-0.205338,-0.090753,0.131469,-0.180991,-0.378897,0.223324,-0.337923,1,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,1460


In [16]:
# from signaturizer import Signaturizer
# # sign = Signaturizer('A1')
# import numpy as np
# np.bool = np.bool_

In [5]:
# impute_layers=["A1","A2","A3","A4","A5","B1","B2","B3","B4","B5",
#                "C1","C2","C3","C4","C5","D1","D2","D3","D4","D5",
#                "E1","E2","E3","E4","E5"]
# data_all=np.empty([df.shape[0],0])
# for layer in impute_layers:
#     print(layer)
#     sign = Signaturizer(layer)
#     data_tmp = sign.predict(df["smiles"])
#     data_tmp_sig=data_tmp.signature
#     data_all=np.concatenate([data_all,data_tmp_sig], axis=1)
#     # convert array into dataframe
#     DF = pd.DataFrame(data_all, index=df["smiles"])
 
#     # save the dataframe as a csv file
#     DF.to_csv("extend_dataset/BBBP/SNN_imputed_BBBP_25layers_v2.csv")
# #                         Tox21_B2.signature,Tox21_B3.signature,
# #                         Tox21_B4.signature,Tox21_B5.signature,
# #                        Tox21_C1.signature,Tox21_C2.signature,Tox21_C3.signature,Tox21_C4.signature,Tox21_C5.signature,
# #                        Tox21_D1.signature,Tox21_D2.signature,Tox21_D3.signature,Tox21_D4.signature,Tox21_D5.signature,
# #                        Tox21_E1.signature,Tox21_E2.signature,Tox21_E3.signature,Tox21_E5.signature], axis=1)


In [6]:
SNN_impute_data = pd.read_csv("extend_dataset/ClinTox/SNN_imputed_ClinTox_25layers_v2.csv")

In [7]:
SNN_impute_data
# only_A1_index=~HIV_data["smiles"].isna()

,smiles,0,1,2,3,4,5,6,7,8,...,3190,3191,3192,3193,3194,3195,3196,3197,3198,3199
0,[C@@H]1([C@@H]([C@@H]([C@H]([C@@H]([C@@H]1Cl)C...,-0.103877,-0.103820,-0.103817,0.097137,-0.081850,0.093517,0.094562,-0.003343,0.097237,...,-0.026632,0.170678,-0.115690,0.130470,0.134450,0.050667,0.078797,-0.020609,0.074398,-0.135756
1,[C@H]([C@@H]([C@@H](C(=O)[O-])O)O)([C@H](C(=O)...,-0.094699,0.020949,0.084003,-0.092960,0.094669,0.053156,0.094697,-0.094700,0.094700,...,-0.018627,-0.081421,0.110827,-0.100026,-0.022824,0.113783,-0.099600,0.109208,0.117016,-0.080514
2,[H]/[NH+]=C(/C1=CC(=O)/C(=C\C=c2ccc(=C([NH3+])...,-0.102093,0.023950,0.005277,0.102015,0.098747,-0.091728,-0.030918,-0.102087,-0.065363,...,-0.026594,0.133172,0.148939,0.125560,0.066672,0.132885,-0.157803,0.009756,0.017042,0.107855
3,[H]/[NH+]=C(\N)/c1ccc(cc1)OCCCCCOc2ccc(cc2)/C(...,-0.097534,0.069429,0.100098,-0.052995,-0.099415,0.101292,-0.085020,0.033412,0.101299,...,0.102308,0.141446,0.070324,0.028403,0.044147,-0.118658,0.074170,-0.125593,-0.073960,-0.127624
4,[N+](=O)([O-])[O-],-0.091662,0.081160,0.043211,0.091662,0.091662,0.091662,0.090463,-0.091662,-0.091662,...,0.072598,-0.039619,-0.072411,0.137696,0.076592,0.113171,0.145429,0.130336,0.112680,0.058564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,O[Si](=O)O,-0.096772,-0.096352,0.006860,0.003662,0.095318,0.096492,0.096662,-0.096723,0.096772,...,-0.090581,0.117562,-0.098291,0.112174,0.090828,0.098788,0.079708,-0.000473,0.099734,-0.198335
1473,O=[Ti]=O,-0.096257,-0.091277,-0.030073,0.065148,0.092341,0.095817,0.094062,-0.096045,0.096246,...,-0.128099,0.032468,-0.154031,0.060419,0.129550,0.071197,-0.045103,0.060664,0.061683,-0.150089
1474,O=[Zn],-0.094213,-0.094166,-0.087980,0.092296,0.080202,0.093980,0.089811,-0.092581,0.094164,...,-0.086052,0.027358,-0.181040,0.006121,0.173035,0.098273,-0.056636,-0.040281,-0.020456,-0.158833
1475,OCl(=O)(=O)=O,-0.097526,-0.095366,-0.071583,0.085996,0.097412,0.096679,0.097099,-0.097521,0.097509,...,-0.107303,-0.014380,-0.080761,0.019285,0.100558,0.035031,-0.101089,0.128816,0.118661,-0.154470


In [8]:
SNN_impute_train_index = pd.read_csv("extend_dataset/ClinTox/ClinTox_train_index_1over4.csv")
SNN_impute_valid_index = pd.read_csv("extend_dataset/ClinTox/ClinTox_valid_index_1over4.csv")
SNN_impute_test_index = pd.read_csv("extend_dataset/ClinTox/ClinTox_test_index.csv")


In [9]:
SNN_impute_train_index=SNN_impute_train_index["0"].values

In [10]:
X_train=SNN_impute_data.iloc[SNN_impute_train_index,1:]

In [12]:

y_train=df["CT_TOX"].values
y_train=y_train[SNN_impute_train_index]

In [13]:
SNN_impute_valid_index

,0
0,3
1,10
2,24
3,38
4,50
...,...
143,1273
144,1278
145,1289
146,1292


In [14]:
SNN_impute_valid_index=SNN_impute_valid_index["0"].values
X_valid=SNN_impute_data.iloc[SNN_impute_valid_index,1:]
y_valid=df["CT_TOX"].values
y_valid=y_valid[SNN_impute_valid_index]



In [15]:
SNN_impute_test_index=SNN_impute_test_index["0"].values
# X=SNN_impute_train_index=SNN_impute_train_index["0"].valuesX[~extend_data["A1.0"].isna()]

In [16]:
X_test=SNN_impute_data.iloc[SNN_impute_test_index,1:]
y_test=df["CT_TOX"].values
y_test=y_test[SNN_impute_test_index]

In [17]:
X_test[1:20]

,0,1,2,3,4,5,6,7,8,9,...,3190,3191,3192,3193,3194,3195,3196,3197,3198,3199
54,-0.101712,0.100133,0.099435,-0.096248,0.101991,-0.043851,0.101956,-0.101991,0.101973,0.101963,...,-0.073534,-0.019519,-0.004900,-0.064997,0.143162,0.053964,-0.057389,-0.068282,0.099844,-0.168496
72,-0.101328,-0.098450,0.091377,-0.102455,-0.041354,0.102512,0.101902,-0.085231,0.102563,0.099630,...,0.025494,0.026854,0.060782,-0.131094,0.056910,0.103575,0.027381,-0.015810,0.015752,-0.010823
73,-0.113751,0.066864,-0.074210,0.110907,-0.107015,-0.094455,0.110975,0.100853,-0.072281,-0.091818,...,0.139775,0.136872,-0.056177,-0.054231,0.036717,0.149636,0.082084,0.044146,-0.049575,-0.143022
80,0.090292,0.091243,-0.086980,-0.091456,0.091474,-0.091472,0.091474,-0.091474,0.091473,0.091472,...,0.123090,-0.074475,0.119177,0.049738,0.039536,0.085139,0.092024,0.083453,0.116921,-0.117065
92,-0.091979,0.087383,-0.036171,0.091980,0.091980,0.091980,-0.027957,-0.091980,-0.091980,-0.091980,...,0.142041,0.039900,-0.037964,0.023608,0.038509,0.086819,0.041923,0.116226,0.057248,-0.136993
94,-0.110161,0.053048,0.017213,0.104253,0.111636,-0.082049,-0.093842,-0.111804,0.096103,0.111588,...,0.112948,-0.101201,0.014298,-0.083521,0.153383,0.149028,0.011913,0.078052,0.032734,-0.174368
125,0.092051,0.100021,-0.041260,-0.100054,0.099613,-0.052712,0.099976,-0.099930,0.100027,0.100058,...,0.055227,0.069874,-0.109708,-0.048371,0.017294,0.103254,0.129099,-0.030534,0.116335,0.004338
166,-0.103124,-0.091029,-0.092915,0.102344,-0.078264,-0.082052,0.085760,-0.103150,0.103149,0.102837,...,0.020597,-0.052106,-0.093419,0.024310,0.007075,0.090733,0.061889,0.090726,-0.102842,0.000960
184,0.038164,-0.064564,0.120069,0.106979,-0.108513,0.120796,-0.071446,0.093637,0.122413,-0.025715,...,-0.059578,0.124312,-0.096810,0.117391,0.038757,-0.060123,-0.121957,0.072402,0.032877,0.057476
188,-0.101712,-0.094377,0.090764,-0.072650,0.061106,0.101744,0.099192,-0.099570,0.101834,0.099078,...,-0.010124,-0.123413,0.083797,-0.035141,0.104241,0.052946,0.061113,0.062117,0.106641,-0.134813


In [16]:
# y=y[~extend_data["A1.0"].isna()]

In [18]:
import numpy as np
from scipy.stats import randint
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn import metrics 
from xgboost import XGBClassifier

names = [
    "Linear SVM",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "XGBoost"
]

classifiers = [
    SVC(kernel="linear", C=0.025),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    AdaBoostClassifier(),
    XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic'),
]

In [19]:

# from sklearn.model_selection import train_test_split
# # i.e. 70 % training dataset and 30 % test datasets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42,stratify=y)
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=0)
# resample_noimpute_train, resample_noimpute_train_y = rus.fit_resample(X_train, y_train)

# #### resample test set
# rus_test = RandomUnderSampler(random_state=0)
# resample_noimpute_test, resample_noimpute_test_y = rus_test.fit_resample(X_test, y_test)


SNN_impute = []
for name, clf in zip(names, classifiers):
        #ax = plt.subplot(len(datasets), len(classifiers) + 1, i)

        clf = make_pipeline(StandardScaler(), clf)
        clf.fit(X_train, y_train)
        ## validation
        y_valid_pred = clf.predict(X_valid)
        
        
        # performing predictions on the test dataset
        y_test_pred = clf.predict(X_test)
        
        ROC_AUC_valid = metrics.roc_auc_score(y_valid, y_valid_pred, multi_class='ovr')
        ROC_AUC_test = metrics.roc_auc_score(y_test, y_test_pred, multi_class='ovr')
        # metrics are used to find accuracy or error
 
        print()
  
        # using metrics module for accuracy calculation
#         print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
        print("Valid ROC-AUC OF THE MODEL: ",ROC_AUC_valid)
        print("Test ROC-AUC OF THE MODEL: ",ROC_AUC_test)
        no_impute_tmp ={"Method":name,
                         "ROC-AUC":ROC_AUC_test, "Dataset":"Tox21", "Input":"SNN Impute"}
        SNN_impute.append(no_impute_tmp)


Valid ROC-AUC OF THE MODEL:  0.5401709401709401
Test ROC-AUC OF THE MODEL:  0.7653743315508021

Valid ROC-AUC OF THE MODEL:  0.5
Test ROC-AUC OF THE MODEL:  0.5

Valid ROC-AUC OF THE MODEL:  0.5517094017094017
Test ROC-AUC OF THE MODEL:  0.7727272727272727

Valid ROC-AUC OF THE MODEL:  0.523931623931624
Test ROC-AUC OF THE MODEL:  0.7690508021390374

Valid ROC-AUC OF THE MODEL:  0.5047008547008547
Test ROC-AUC OF THE MODEL:  0.4963235294117647


In [20]:
snn_df=pd.DataFrame(SNN_impute)
snn_df["ROC-AUC"].mean()

0.6606951871657755

In [21]:
tox21_data=pd.read_csv("extend_dataset/ClinTox/ClinTox_train_1over4.csv")

# tox21_data=tox21_data[~tox21_data["A1.0"].isna()]
Tox21_ourimpute_train=tox21_data.iloc[:,2:3202].fillna(0.0).values
Tox21_ourimpute_train_y=tox21_data.loc[:,"CT_TOX"].values

tox21_data_valid=pd.read_csv("extend_dataset/ClinTox/ClinTox_valid_1over4.csv")

# tox21_data=tox21_data[~tox21_data["A1.0"].isna()]
Tox21_ourimpute_valid=tox21_data_valid.iloc[:,2:3202].fillna(0.0).values
Tox21_ourimpute_valid_y=tox21_data_valid.loc[:,"CT_TOX"].values


tox21_data_test=pd.read_csv("extend_dataset/ClinTox/ClinTox_test.csv")

# tox21_data=tox21_data[~tox21_data["A1.0"].isna()]
Tox21_ourimpute_test=tox21_data_test.iloc[:,2:3202].fillna(0.0).values
Tox21_ourimpute_test_y=tox21_data_test.loc[:,"CT_TOX"].values

In [22]:

X_train=Tox21_ourimpute_train[:,0:128]
y_train=Tox21_ourimpute_train_y

X_valid=Tox21_ourimpute_valid[:,0:128]
y_valid=Tox21_ourimpute_valid_y

X_test=Tox21_ourimpute_test[:,0:128]
y_test=Tox21_ourimpute_test_y


In [23]:
# from sklearn.model_selection import train_test_split
# # i.e. 70 % training dataset and 30 % test datasets
# X_train, X_test, y_train, y_test = train_test_split(X_SNN_A1, y_SNN_A1, test_size = 0.30, random_state=42,stratify=y_SNN_A1)

In [24]:
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=0)
# resample_noimpute_train, resample_noimpute_train_y = rus.fit_resample(X_train, y_train)

In [25]:

no_impute = []
for name, clf in zip(names, classifiers):
        #ax = plt.subplot(len(datasets), len(classifiers) + 1, i)

        clf = make_pipeline(StandardScaler(), clf)
        clf.fit(X_train,y_train)
        ## validation
        y_valid_pred = clf.predict(X_valid)
        
        
        # performing predictions on the test dataset
        y_test_pred = clf.predict(X_test)
        
        ROC_AUC_valid = metrics.roc_auc_score(y_valid, y_valid_pred, multi_class='ovr')
        ROC_AUC_test = metrics.roc_auc_score(y_test, y_test_pred, multi_class='ovr')
        # metrics are used to find accuracy or error
 
        print()
  
        # using metrics module for accuracy calculation
#         print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
        print("Valid ROC-AUC OF THE MODEL: ",ROC_AUC_valid)
        print("Test ROC-AUC OF THE MODEL: ",ROC_AUC_test)
        no_impute_tmp ={"Method":name,
                         "ROC-AUC":ROC_AUC_test, "Dataset":"Tox21", "Input":"SNN Impute"}
        no_impute.append(no_impute_tmp)
        
        


Valid ROC-AUC OF THE MODEL:  0.5763105507631056
Test ROC-AUC OF THE MODEL:  0.6176470588235294

Valid ROC-AUC OF THE MODEL:  0.5
Test ROC-AUC OF THE MODEL:  0.5

Valid ROC-AUC OF THE MODEL:  0.7435301924353018
Test ROC-AUC OF THE MODEL:  0.6482843137254901

Valid ROC-AUC OF THE MODEL:  0.5726609157266093
Test ROC-AUC OF THE MODEL:  0.5649509803921569

Valid ROC-AUC OF THE MODEL:  0.4927007299270073
Test ROC-AUC OF THE MODEL:  0.5379901960784315


In [26]:
no_impute_df=pd.DataFrame(no_impute)
no_impute_df["ROC-AUC"].mean()

0.5737745098039216